In [1]:
import os as os
import numpy as np
import pandas as pd
import polars as pl
import tqdm
import slippi as slp
from joblib import Parallel, delayed
from multiprocessing import Manager
import time

<h2> Preliminary Functions </h2>

We use these functions to one-hot encode the button bitmask and get the frame data for a given port number and frames object.

In [2]:
# Set the number of time steps in the model inputs
frames_per_input = 60 * 12  # 12 seconds of gameplay

# Function to one-hot encode controller bitmask
def one_hot_encode(bitmask):
    labels = ['DPAD_LEFT', 'DPAD_RIGHT', 'DPAD_DOWN', 'DPAD_UP', 'Z', 'R', 'L', 'A', 'B', 'X', 'Y', 'START']
    encoded_values = [1, 2, 4, 8, 16, 32, 64, 256, 512, 1024, 2048, 4096]

    # Create a dictionary mapping labels to their encoded values
    label_to_value = dict(zip(labels, encoded_values))

    # Initialize a list to store the one-hot encoded values
    one_hot_encoded = [0] * len(labels)

    # Iterate through labels and set the corresponding one-hot encoded value
    for label, value in label_to_value.items():
        if bitmask & value:
            one_hot_encoded[labels.index(label)] = 1

    return one_hot_encoded

# Function to get frame data for a given set of frames and port
def get_frame_inputs(frames, port):
    sheik_inputs = np.empty((len(frames), 18))  # Initialize an empty NumPy array    
    for i, frame in enumerate(frames):   # Takes all the frames, skipping the first 5 seconds.
        buttons = one_hot_encode(frame.ports[port].leader.pre.buttons.physical.value)
        j_x = frame.ports[port].leader.pre.joystick.x
        j_y = frame.ports[port].leader.pre.joystick.y
        c_x = frame.ports[port].leader.pre.cstick.x
        c_y = frame.ports[port].leader.pre.cstick.y
        t_l = frame.ports[port].leader.pre.triggers.physical.l
        t_r = frame.ports[port].leader.pre.triggers.physical.r

        frame_data = buttons + [j_x, j_y, c_x, c_y, t_l, t_r]
        sheik_inputs[i] = frame_data

    return sheik_inputs





In [3]:




# Function to process a single SLP file and append to shared polars df
def process_slp_file(slp_file, dataset_path,game_data_list):#,df):#, time_series_list, label_list, ids):
    try:
        file_path = os.path.join(dataset_path, slp_file)
        game = slp.Game(file_path)
        frames = game.frames

        if len(frames) <  + frames_per_input:  # Ignore games that are <3600 frames (i.e. <60 seconds)
            return

        # List occupied ports
        occupied_ports = [i for i, port in enumerate(game.start.players) if port is not None]
        port_1 = occupied_ports[0]
        port_2 = occupied_ports[1]

        if len(occupied_ports) > 2:  # Ignore games that aren't singles
            return
        
        # port_1_frame_
        port_1_frame_inputs = get_frame_inputs(frames, port_1)
        port_2_frame_inputs = get_frame_inputs(frames, port_2)
        
        game_data_dict = {
            # Start (Game Data)
            'file': slp_file,
            # 'is_teams': game.start.stage.is_teams, # bool
            # 'players': game.start.stage.players, #tuple
            # 'random_seed':game.start.stage.random_seed # int
            # 'slippi': game.start.stage.slippi #Slippi
            'stage_name': game.start.stage.name,
            'stage_value': game.start.stage.value,
            'is_pal': game.start.is_pal,
            'is_frozen_ps': game.start.is_frozen_ps,
            # Player 1 Data
            'port_1': port_1,
            'port_1_character_name': game.start.players[port_1].character.name,
            'port_1_character_value': game.start.players[port_1].character.value,
            'port_1_type_name': game.start.players[port_1].type.name,
            'port_1_type_value': game.start.players[port_1].type.value,
            'port_1_stocks': game.start.players[port_1].stocks,
            'port_1_costume': game.start.players[port_1].costume,
            # 'port_1_team': game.start.players[port_1].team # team|None
            'port_1_ucf_dash_back_name': game.start.players[port_1].ucf.dash_back.name,
            'port_1_ucf_dash_back_value': game.start.players[port_1].ucf.dash_back.value,
            'port_1_ucf_shield_drop_name': game.start.players[port_1].ucf.shield_drop.name,
            'port_1_ucf_shield_drop_value': game.start.players[port_1].ucf.shield_drop.value,
            'port_1_tag': game.start.players[port_1].tag,
            # Player 2 Data
            'port_2': port_2,
            'port_2_character_name': game.start.players[port_2].character.name,
            'port_2_character_value': game.start.players[port_2].character.value,
            'port_2_type_name': game.start.players[port_2].type.name,
            'port_2_type_value': game.start.players[port_2].type.value,
            'port_2_stocks': game.start.players[port_2].stocks,
            'port_2_costume': game.start.players[port_2].costume,
            # 'port_2_team': game.start.players[port_2].team # team|None
            'port_2_ucf_dash_back_name': game.start.players[port_2].ucf.dash_back.name,
            'port_2_ucf_dash_back_value': game.start.players[port_2].ucf.dash_back.value,
            'port_2_ucf_shield_drop_name': game.start.players[port_2].ucf.shield_drop.name,
            'port_2_ucf_shield_drop_value': game.start.players[port_2].ucf.shield_drop.value,
            'port_2_tag': game.start.players[port_2].tag,
            # End
            'method_how_game_ended': game.end.method, #string
            # Player 1 Pre Frame data
            'port_1_pre_state_value': [np.array([frame.ports[port_1].leader.pre.state.value if frame.ports[port_1].leader.pre.state else None for frame in frames[:]])], #object
            'port_1_pre_state_name': [np.array([frame.ports[port_1].leader.pre.state.name if frame.ports[port_1].leader.pre.state else None for frame in frames[:]])], #object
            'port_1_pre_position_x': [np.array([frame.ports[port_1].leader.pre.position.x for frame in frames[:]])], #object
            'port_1_pre_position_y': [np.array([frame.ports[port_1].leader.pre.position.y for frame in frames[:]])], #object
            'port_1_pre_direction': [np.array([frame.ports[port_1].leader.pre.direction for frame in frames[:]])], #object
            'port_1_pre_damage':[np.array([frame.ports[port_1].leader.pre.damage for frame in frames[:]])], #object
            # Player 1 Frame pre Inputs
            'port_1_DPAD_LEFT': [port_1_frame_inputs[:, 0]],
            'port_1_DPAD_RIGHT': [port_1_frame_inputs[:, 1]],
            'port_1_DPAD_DOWN': [port_1_frame_inputs[:, 2]],
            'port_1_DPAD_UP': [port_1_frame_inputs[:, 3]],
            'port_1_Z': [port_1_frame_inputs[:, 4]],
            'port_1_R': [port_1_frame_inputs[:, 5]],
            'port_1_L': [port_1_frame_inputs[:, 6]],
            'port_1_A': [port_1_frame_inputs[:, 7]],
            'port_1_B': [port_1_frame_inputs[:, 8]],
            'port_1_X': [port_1_frame_inputs[:, 9]],
            'port_1_Y': [port_1_frame_inputs[:, 10]],
            'port_1_START': [port_1_frame_inputs[:, 11]],
            'port_1_J_X': [port_1_frame_inputs[:, 12]],
            'port_1_J_Y': [port_1_frame_inputs[:, 13]],
            'port_1_C_X': [port_1_frame_inputs[:, 14]],
            'port_1_C_Y': [port_1_frame_inputs[:, 15]],
            'port_1_T_L': [port_1_frame_inputs[:, 16]],
            'port_1_T_R': [port_1_frame_inputs[:, 17]],
            # Player 1 post frame data
            'port_1_post_character_value': [np.array([frame.ports[port_1].leader.post.character.value if frame.ports[port_1].leader.post.character else None for frame in frames[:]])], #object
            'port_1_post_character_name': [np.array([frame.ports[port_1].leader.post.character.name if frame.ports[port_1].leader.post.character else None for frame in frames[:]])], #object
            'port_1_post_state_value': [np.array([frame.ports[port_1].leader.post.state.value if frame.ports[port_1].leader.post.state else None for frame in frames[:]])], #object
            'port_1_post_state_name': [np.array([frame.ports[port_1].leader.post.state.name if frame.ports[port_1].leader.post.state else None for frame in frames[:]])], #object
            'port_1_post_position_x': [np.array([frame.ports[port_1].leader.post.position.x for frame in frames[:]])], #object
            'port_1_post_position_y': [np.array([frame.ports[port_1].leader.post.position.y for frame in frames[:]])], #object
            'port_1_post_direction': [np.array([frame.ports[port_1].leader.post.direction for frame in frames[:]])], #object
            'port_1_post_damage': [np.array([frame.ports[port_1].leader.post.damage for frame in frames[:]])], #object
            'port_1_post_shield': [np.array([frame.ports[port_1].leader.post.shield for frame in frames[:]])], #object
            'port_1_post_stocks': [np.array([frame.ports[port_1].leader.post.stocks for frame in frames[:]])], #object
            'port_1_post_last_attack_landed_value': [np.array([frame.ports[port_1].leader.post.last_attack_landed.value if frame.ports[port_1].leader.post.last_attack_landed else None for frame in frames[:]])], #object
            'port_1_post_last_attack_landed_name': [np.array([frame.ports[port_1].leader.post.last_attack_landed.name if frame.ports[port_1].leader.post.last_attack_landed else None for frame in frames[:]])], #object
            # 'port_1_post_last_hit_by': [np.array([frame.ports[port_1].leader.post.last_hit_by for frame in frames[:]])], #object
            'port_1_post_combo_count': [np.array([frame.ports[port_1].leader.post.combo_count for frame in frames[:]])], #object
            # 'port_1_post_state_age': [np.array([frame.ports[port_1].leader.post.state_age for frame in frames[:]])], #object
            'port_1_post_flags_value': [np.array([frame.ports[port_1].leader.post.flags.value for frame in frames[:]])], #object
            'port_1_post_hit_stun': [np.array([frame.ports[port_1].leader.post.hit_stun for frame in frames[:]])], #object ??????
            'port_1_post_airborne': [np.array([frame.ports[port_1].leader.post.airborne for frame in frames[:]])], #object
            'port_1_post_ground': [np.array([frame.ports[port_1].leader.post.ground for frame in frames[:]])], #object
            'port_1_post_jumps': [np.array([frame.ports[port_1].leader.post.jumps for frame in frames[:]])], #object
            'port_1_post_l_cancel': [np.array([frame.ports[port_1].leader.post.l_cancel for frame in frames[:]])], #object
             # Player 2 Pre Frame data
            'port_2_pre_state_value': [np.array([frame.ports[port_2].leader.pre.state.value if frame.ports[port_2].leader.pre.state else None for frame in frames[:]])], #object
            'port_2_pre_state_name': [np.array([frame.ports[port_2].leader.pre.state.name if frame.ports[port_2].leader.pre.state else None for frame in frames[:]])], #object
            'port_2_pre_position_x': [np.array([frame.ports[port_2].leader.pre.position.x for frame in frames[:]])], #object
            'port_2_pre_position_y': [np.array([frame.ports[port_2].leader.pre.position.y for frame in frames[:]])], #object
            'port_2_pre_direction': [np.array([frame.ports[port_2].leader.pre.direction for frame in frames[:]])], #object
            'port_2_pre_damage':[np.array([frame.ports[port_2].leader.pre.damage for frame in frames[:]])], #object
            # Player 2 Frame pre Inputs
            'port_2_DPAD_LEFT': [port_2_frame_inputs[:, 0]],
            'port_2_DPAD_RIGHT': [port_2_frame_inputs[:, 1]],
            'port_2_DPAD_DOWN': [port_2_frame_inputs[:, 2]],
            'port_2_DPAD_UP': [port_2_frame_inputs[:, 3]],
            'port_2_Z': [port_2_frame_inputs[:, 4]],
            'port_2_R': [port_2_frame_inputs[:, 5]],
            'port_2_L': [port_2_frame_inputs[:, 6]],
            'port_2_A': [port_2_frame_inputs[:, 7]],
            'port_2_B': [port_2_frame_inputs[:, 8]],
            'port_2_X': [port_2_frame_inputs[:, 9]],
            'port_2_Y': [port_2_frame_inputs[:, 10]],
            'port_2_START': [port_2_frame_inputs[:, 11]],
            'port_2_J_X': [port_2_frame_inputs[:, 12]],
            'port_2_J_Y': [port_2_frame_inputs[:, 13]],
            'port_2_C_X': [port_2_frame_inputs[:, 14]],
            'port_2_C_Y': [port_2_frame_inputs[:, 15]],
            'port_2_T_L': [port_2_frame_inputs[:, 16]],
            'port_2_T_R': [port_2_frame_inputs[:, 17]],
            # Player 2 post frame data
            'port_2_post_character_value': [np.array([frame.ports[port_2].leader.post.character.value if frame.ports[port_2].leader.post.character else None for frame in frames[:]])], #object
            'port_2_post_character_name': [np.array([frame.ports[port_2].leader.post.character.name if frame.ports[port_2].leader.post.character else None for frame in frames[:]])], #object
            'port_2_post_state_value': [np.array([frame.ports[port_2].leader.post.state.value if frame.ports[port_2].leader.post.state else None for frame in frames[:]])], #object
            'port_2_post_state_name': [np.array([frame.ports[port_2].leader.post.state.name if frame.ports[port_2].leader.post.state else None for frame in frames[:]])], #object
            'port_2_post_position_x': [np.array([frame.ports[port_2].leader.post.position.x for frame in frames[:]])], #object
            'port_2_post_position_y': [np.array([frame.ports[port_1].leader.post.position.y for frame in frames[:]])], #object
            'port_2_post_direction': [np.array([frame.ports[port_2].leader.post.direction for frame in frames[:]])], #object
            'port_2_post_damage': [np.array([frame.ports[port_2].leader.post.damage for frame in frames[:]])], #object
            'port_2_post_shield': [np.array([frame.ports[port_2].leader.post.shield for frame in frames[:]])], #object
            'port_2_post_stocks': [np.array([frame.ports[port_2].leader.post.stocks for frame in frames[:]])], #object
            'port_2_post_last_attack_landed_value': [np.array([frame.ports[port_2].leader.post.last_attack_landed.value if frame.ports[port_2].leader.post.last_attack_landed else None for frame in frames[:]])], #object
            'port_2_post_last_attack_landed_name': [np.array([frame.ports[port_2].leader.post.last_attack_landed.name if frame.ports[port_2].leader.post.last_attack_landed else None for frame in frames[:]])], #object
            # 'port_2_post_last_hit_by': [np.array([frame.ports[port_2].leader.post.last_hit_by for frame in frames[:]])], #object
            'port_2_post_combo_count': [np.array([frame.ports[port_2].leader.post.combo_count for frame in frames[:]])], #object
            # 'port_2_post_state_age': [np.array([frame.ports[port_2].leader.post.state_age for frame in frames[:]])], #object
            'port_2_post_flags_value': [np.array([frame.ports[port_2].leader.post.flags.value for frame in frames[:]])], #object
            'port_2_post_hit_stun': [np.array([frame.ports[port_2].leader.post.hit_stun for frame in frames[:]])], #object ??????
            'port_2_post_airborne': [np.array([frame.ports[port_2].leader.post.airborne for frame in frames[:]])], #object
            'port_2_post_ground': [np.array([frame.ports[port_2].leader.post.ground for frame in frames[:]])], #object
            'port_2_post_jumps': [np.array([frame.ports[port_2].leader.post.jumps for frame in frames[:]])], #object
            'port_2_post_l_cancel': [np.array([frame.ports[port_2].leader.post.l_cancel for frame in frames[:]])], #object
            #Item
            # 'item_':[np.array([frame.Item.type.value for frame in frames[:]])]# I CANT FIGURE THIS ONE OUT
            # 'item_state':[np.array([frame.Item.state for frame in frames[:]])],# object
            # 'item_direction':[np.array([frame.Item.direction for frame in frames[:]])],# object
            # 'item_velocity_x':[np.array([frame.Item.velocity.x for frame in frames[:]])],# object
            # 'item_velocity_y':[np.array([frame.Item.velocity.y for frame in frames[:]])],# object
            # 'item_position_x':[np.array([frame.Item.position.x for frame in frames[:]])],# object
            # 'item_position_y':[np.array([frame.Item.position.y for frame in frames[:]])],# object
            # 'item_damage':[np.array([frame.Item.damage for frame in frames[:]])],# object
            # 'item_timer':[np.array([frame.Item.timer for frame in frames[:]])],# object
            # 'item_spawn_id':[np.array([frame.Item.spawn_id for frame in frames[:]])],# object
            
        }
        game_data_list.append(game_data_dict.values())
        return game_data_dict
    except Exception as e:
        print(f"Error processing {slp_file}: {str(e)}")
        


    
# Create an empty Polars DataFrame with the desired schema
schema =    [
    ('file', pl.String),
    ('stage_name', pl.String),
    ('stage_value', pl.Int8),
    ('is_pal', pl.Boolean),
    ('is_frozen_ps', pl.Boolean),
    # Player 1 Data
    ('port_1', pl.Int32),
    ('port_1_character_name', pl.String),
    ('port_1_character_value', pl.Int32),
    ('port_1_type_name', pl.String),
    ('port_1_type_value', pl.Boolean),
    ('port_1_stocks', pl.Int32),
    ('port_1_costume', pl.Int32),
    ('port_1_ucf_dash_back_name', pl.String),
    ('port_1_ucf_dash_back_value', pl.Int32),
    ('port_1_ucf_shield_drop_name', pl.String),
    ('port_1_ucf_shield_drop_value', pl.Int32),
    ('port_1_tag', pl.String),
    # Player 2 Data
    ('port_2', pl.Int32),
    ('port_2_character_name', pl.String),
    ('port_2_character_value', pl.Int32),
    ('port_2_type_name', pl.String),
    ('port_2_type_value', pl.Boolean),
    ('port_2_stocks', pl.Int32),
    ('port_2_costume', pl.Int32),
    ('port_2_ucf_dash_back_name', pl.String),
    ('port_2_ucf_dash_back_value', pl.Int32),
    ('port_2_ucf_shield_drop_name', pl.String),
    ('port_2_ucf_shield_drop_value', pl.Int32),
    ('port_2_tag', pl.String),
     # End
    ('method_how_game_ended',pl.String),
    #  # Player 1 Pre Frame data
    ('port_1_pre_state_value',pl.Object),
    ('port_1_pre_state_name',pl.Object),
    ('port_1_pre_position_x',pl.Object),
    ('port_1_pre_position_y',pl.Object),
    ('port_1_pre_direction',pl.Object),
    ('port_1_pre_damage',pl.Object),
        # pre Input Data for Player 1
    ('port_1_DPAD_LEFT', pl.Object),
    ('port_1_DPAD_RIGHT', pl.Object),
    ('port_1_DPAD_DOWN', pl.Object),
    ('port_1_DPAD_UP', pl.Object),
    ('port_1_Z', pl.Object),
    ('port_1_R', pl.Object),
    ('port_1_L', pl.Object),
    ('port_1_A', pl.Object),
    ('port_1_B', pl.Object),
    ('port_1_X', pl.Object),
    ('port_1_Y', pl.Object),
    ('port_1_START', pl.Object),
    ('port_1_J_X', pl.Object),
    ('port_1_J_Y', pl.Object),
    ('port_1_C_X', pl.Object),
    ('port_1_C_Y', pl.Object),
    ('port_1_T_L', pl.Object),
    ('port_1_T_R', pl.Object),
    # Player 1 post frame data
    ('port_1_post_character_value', pl.Object),
    ('port_1_post_character_name', pl.Object),
    ('port_1_post_state_value', pl.Object),
    ('port_1_post_state_name', pl.Object),
    ('port_1_post_position_x', pl.Object),
    ('port_1_post_position_y', pl.Object),
    ('port_1_post_direction', pl.Object),
    ('port_1_post_damage', pl.Object),
    ('port_1_post_shield', pl.Object),
    ('port_1_post_stocks', pl.Object),
    ('port_1_post_last_attack_landed_value', pl.Object),
    ('port_1_post_last_attack_landed_name', pl.Object),
    ('port_1_post_combo_count', pl.Object),
    ('port_1_post_flags_value', pl.Object),
    ('port_1_post_hit_stun', pl.Object),
    ('port_1_post_airborne', pl.Object),
    ('port_1_post_ground', pl.Object),
    ('port_1_post_jumps', pl.Object),
    ('port_1_post_l_cancel', pl.Object),
    # Player 1 Pre Frame data
    ('port_2_pre_state_value',pl.Object),
    ('port_2_pre_state_name',pl.Object),
    ('port_2_pre_position_x',pl.Object),
    ('port_2_pre_position_y',pl.Object),
    ('port_2_pre_direction',pl.Object),
    ('port_2_pre_damage',pl.Object),
    # pre Input Data for Player 1
    ('port_2_DPAD_LEFT', pl.Object),
    ('port_2_DPAD_RIGHT', pl.Object),
    ('port_2_DPAD_DOWN', pl.Object),
    ('port_2_DPAD_UP', pl.Object),
    ('port_2_Z', pl.Object),
    ('port_2_R', pl.Object),
    ('port_2_L', pl.Object),
    ('port_2_A', pl.Object),
    ('port_2_B', pl.Object),
    ('port_2_X', pl.Object),
    ('port_2_Y', pl.Object),
    ('port_2_START', pl.Object),
    ('port_2_J_X', pl.Object),
    ('port_2_J_Y', pl.Object),
    ('port_2_C_X', pl.Object),
    ('port_2_C_Y', pl.Object),
    ('port_2_T_L', pl.Object),
    ('port_2_T_R', pl.Object),
    # Player 1 post frame data
    ('port_2_post_character_value', pl.Object),
    ('port_2_post_character_name', pl.Object),
    ('port_2_post_state_value', pl.Object),
    ('port_2_post_state_name', pl.Object),
    ('port_2_post_position_x', pl.Object),
    ('port_2_post_position_y', pl.Object),
    ('port_2_post_direction', pl.Object),
    ('port_2_post_damage', pl.Object),
    ('port_2_post_shield', pl.Object),
    ('port_2_post_stocks', pl.Object),
    ('port_2_post_last_attack_landed_value', pl.Object),
    ('port_2_post_last_attack_landed_name', pl.Object),
    ('port_2_post_combo_count', pl.Object),
    ('port_2_post_flags_value', pl.Object),
    ('port_2_post_hit_stun', pl.Object),
    ('port_2_post_airborne', pl.Object),
    ('port_2_post_ground', pl.Object),
    ('port_2_post_jumps', pl.Object),
    ('port_2_post_l_cancel', pl.Object),

    # Item (You can uncomment and add these fields if needed)
    # ('item_type_value', pl.Object),
    # ('item_state', pl.Object),
    # ('item_direction', pl.Object),
    # ('item_velocity_x', pl.Object),
    # ('item_velocity_y', pl.Object),
    # ('item_position_x', pl.Object),
    # ('item_position_y', pl.Object),
    # ('item_damage', pl.Object),
    # ('item_timer', pl.Object),
    # ('item_spawn_id', pl.Object),
]



In [4]:
# # Set the number of time steps in the model inputs
# frames_per_input = 60 * 12  # 12 seconds of gameplay

# dataset_path = './Slippi_Public_Dataset_v3/'

# # List of file names
# slp_files = [file for file in os.listdir(dataset_path)]

# # slp_file = slp_files[10]

# # A single file path
# # file_path = os.path.join(dataset_path, slp_file)
# num_files = 1000

# # game_data_df = pl.DataFrame([], schema=schema)
# manager = Manager()
# game_data_list = manager.list()
# # Use joblib to parallelize processing of SLP files
# Parallel(n_jobs=-1, verbose=10)(delayed(process_slp_file)(slp_file, dataset_path, game_data_list) for slp_file in tqdm.tqdm(slp_files[:num_files]))

# # game_data_df = pl.DataFrame(game_data_list, schema=schema)# Convert the ListProxy to a regular list
# game_data_list_converted = list(game_data_list)

# # Now you can create the Polars DataFrame from the converted list
# game_data_df = pl.DataFrame(game_data_list_converted, schema=schema)


# # print(game_data_dict)
# # game_data_new_row = pl.DataFrame(game_data_dict,schema=schema)
# # print(type(game_data_new_row))

# print(game_data_df)
# print()

In [5]:
# Assuming `game_data_df` is your Polars DataFrame
# game_data_df.write_ipc('./data/polars_data_frame.ipc')

In [6]:
# df = pl.DataFrame(
#     {
#         "x": list(reversed(range(1_000_000))),
#         "y": [v / 1000 for v in range(1_000_000)],
#         "z": [str(v) for v in range(1_000_000)],
#     },
#     schema=[("x", pl.UInt32), ("y", pl.Float64), ("z", pl.String)],
# )
# df


In [12]:
# file_path='./data/test_replay.slp'
file_path='C:/Users/jaspa/Grant ML/Data/ranked/ranked-anonymized-1-116248/ranked-anonymized/diamond-diamond-000d05dda0c0561fd901b3e2.slp'
# C:\Users\jaspa\Grant ML\Data\ranked\ranked-anonymized-1-116248\ranked-anonymized

# dataset_path = './Slippi_Public_Dataset_v3/'
# slp_files = [file for file in os.listdir(dataset_path)]
# slp_file = slp_files[0]
# file_path = os.path.join(dataset_path, slp_file)

game = slp.Game(file_path)
# 
# from slippi.parse import parse
# from slippi.parse import ParseEvent

# # handlers = {ParseEvent.METADATA: print}
# # parse(file_path, handlers)

# # handlers = {ParseEvent.METADATA_RAW: print}
# # parse(file_path, handlers)

# handlers = {ParseEvent.START: print}
# parse(file_path, handlers)

# # # handlers = {ParseEvent.FRAME: print}
# # # parse(file_path, handlers)

# # handlers = {ParseEvent.END: print}
# # parse(file_path, handlers)

# # handlers = {ParseEvent.FRAME_START: print}
# # parse(file_path, handlers)

# # handlers = {ParseEvent.ITEM: print}
# # parse(file_path, handlers)

# # handlers = {ParseEvent.FRAME_END: print}
# # parse(file_path, handlers)


# # handlers = {ParseEvent.FRAME: print}
# # parse(file_path, handlers)

ParseError: Parse error (C:\Users\jaspa\Grant ML\Data\ranked\ranked-anonymized-1-116248\ranked-anonymized\diamond-diamond-000d05dda0c0561fd901b3e2.slp @0x22c673): [Errno None] None: 'C:\\Users\\jaspa\\Grant ML\\Data\\ranked\\ranked-anonymized-1-116248\\ranked-anonymized\\diamond-diamond-000d05dda0c0561fd901b3e2.slp'

In [8]:
dataset_path = './Slippi_Public_Dataset_v3/'

# List of file names
slp_files = [file for file in os.listdir(dataset_path)]

# slp_file = slp_files[10]

# A single file path
# file_path = os.path.join(dataset_path, slp_file)
num_files = 10

t_1 = time.time()
for slp_file in slp_files[:num_files]:
    file_path = os.path.join(dataset_path, slp_file)
    game = slp.Game(file_path)
    frames = game.frames
t_2 = time.time()

print(t_2-t_1)

1.734147548675537


In [9]:
dataset_path = './Slippi_Public_Dataset_v3/'

# List of file names
slp_files = [file for file in os.listdir(dataset_path)]

# slp_file = slp_files[10]

# A single file path
# file_path = os.path.join(dataset_path, slp_file)
num_files = 10

t_1 = time.time()
for slp_file in slp_files[:num_files]:
    file_path = os.path.join(dataset_path, slp_file)
    game = slp.Game(file_path)
    frames = game.frames
    occupied_ports = [i for i, port in enumerate(game.start.players) if port is not None]
    port_1 = occupied_ports[0]
    port_2 = occupied_ports[1]
    port_1_post_character_value = np.array([frame.ports[port_1].leader.post.character.value if frame.ports[port_1].leader.post.character else None for frame in frames[:]])
    
t_2 = time.time()

print(t_2-t_1)

2.2049453258514404


In [10]:
from slippi import parse
from slippi.event import Frame, Start
import time
import numpy as np

# Event handler for game start
def handle_game_start(event):
    global occupied_ports
    occupied_ports = [i for i, port in enumerate(event.players) if port is not None]

# Event handler for frame data
def handle_frame(event):
    global port_1_post_character_value, port_1, port_2
    if port_1 in occupied_ports:
        character = event.ports[port_1].leader.post.character
        if character:
            port_1_post_character_value.append(character.value)
    # Add similar logic for port_2 if needed

# Initialize variables
occupied_ports = []
port_1_post_character_value = []
port_1, port_2 = None, None

# Register event handlers
handlers = {
    Start: handle_game_start,
    Frame: handle_frame,
}

# Timing the parsing process
t_1 = time.time()

# Assuming slp_files is a list of file names and dataset_path is defined
for slp_file in slp_files[:num_files]:
    file_path = os.path.join(dataset_path, slp_file)
    parse(file_path, handlers)  # Parse the file with registered event handlers

    # After parsing one file, you can process the collected data as needed
    # Reset or process the collected data for each file as necessary

t_2 = time.time()
print(t_2 - t_1)

# Note: You might need to adjust this example based on the exact structure of your data and what you want to extract.


1.2418453693389893


In [11]:
import os
import numpy as np
import pandas as pd
from slippi import Game
from slippi.event import Start, Frame
from joblib import Parallel, delayed
from multiprocessing import Manager
import tqdm

# frames_per_input = 60 * 12  # 12 seconds of gameplay

# One-hot encoding function remains the same
def one_hot_encode(bitmask):
    labels = ['DPAD_LEFT', 'DPAD_RIGHT', 'DPAD_DOWN', 'DPAD_UP', 'Z', 'R', 'L', 'A', 'B', 'X', 'Y', 'START']
    encoded_values = [1, 2, 4, 8, 16, 32, 64, 256, 512, 1024, 2048, 4096]

    # Create a dictionary mapping labels to their encoded values
    label_to_value = dict(zip(labels, encoded_values))

    # Initialize a list to store the one-hot encoded values
    one_hot_encoded = [0] * len(labels)

    # Iterate through labels and set the corresponding one-hot encoded value
    for label, value in label_to_value.items():
        if bitmask & value:
            one_hot_encoded[labels.index(label)] = 1

    return one_hot_encoded

# Handler function to process frame data
def process_frame_data(frame, port):
    buttons = one_hot_encode(frame.ports[port].leader.pre.buttons.physical.value)
    j_x = frame.ports[port].leader.pre.joystick.x
    j_y = frame.ports[port].leader.pre.joystick.y
    c_x = frame.ports[port].leader.pre.cstick.x
    c_y = frame.ports[port].leader.pre.cstick.y
    t_l = frame.ports[port].leader.pre.triggers.physical.l
    t_r = frame.ports[port].leader.pre.triggers.physical.r
    return buttons + [j_x, j_y, c_x, c_y, t_l, t_r]

# Function to process a single SLP file using event-based parsing
def process_slp_file_event_based(slp_file, dataset_path, time_series_list, label_list, ids):
    try:
        file_path = os.path.join(dataset_path, slp_file)
        game_data = []

        def handle_frame(event, port):
            if event.index > 300:  # Skip the first 5 seconds (300 frames)
                frame_data = process_frame_data(event, port)
                game_data.append(frame_data)

        def parse_game(file_path, port):
            Game(file_path, handlers={Frame: lambda event: handle_frame(event, port)})

        # Parse the game for each player
        game = Game(file_path)
        occupied_ports = [i for i, port in enumerate(game.start.players) if port is not None]
        if len(occupied_ports) != 2:  # Ensure it's a singles game
            return

        for port in occupied_ports:
            parse_game(file_path, port)
            character = game.start.players[port].character.name
            time_series_list.append(np.array(game_data[300: ]))  # Adjust as needed
            label_list.append(1 if character == 'SHEIK' else 0)
            ids.append(slp_file)

    except Exception as e:
        print(f"Error processing {slp_file}: {str(e)}")

# Example usage with parallel processing
dataset_path = './Slippi_Public_Dataset_v3/'
slp_files = [file for file in os.listdir(dataset_path) if file.endswith('.slp') and 'Sheik' in file and 'Fox' in file]
# print(len(slp_files))

# slp_files = [file for file in os.listdir(dataset_path) if file.endswith('.slp')]

# Initialize multiprocessing lists
manager = Manager()
time_series_list = manager.list()
label_list = manager.list()
ids = manager.list()

# Process files in parallel
Parallel(n_jobs=-1, verbose=10)(delayed(process_slp_file_event_based)(slp_file, dataset_path, time_series_list, label_list, ids) for slp_file in tqdm.tqdm(slp_files))

# Convert results to DataFrame
df = pd.DataFrame({"TimeSeries": list(time_series_list), "Label": list(label_list), "FName": list(ids)})
df.sort_values(by=['FName', 'Label'], inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)


  2%|▏         | 48/2362 [00:00<00:23, 98.64it/s][Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    0.7s
  4%|▍         | 96/2362 [00:01<00:27, 81.84it/s][Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:    1.2s
  5%|▌         | 120/2362 [00:01<00:27, 81.82it/s][Parallel(n_jobs=-1)]: Done  80 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:    1.7s
 10%|█         | 240/2362 [00:02<00:26, 81.26it/s][Parallel(n_jobs=-1)]: Done 194 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 217 tasks      | elapsed:    3.2s
100%|██████████| 2362/2362 [00:32<00:00, 73.33it/s]


Empty DataFrame
Columns: [TimeSeries, Label, FName]
Index: []


[Parallel(n_jobs=-1)]: Done 2362 out of 2362 | elapsed:   33.2s finished


In [ ]:
# Function to process a single SLP file using event-based parsing
def process_slp_file_event_based(slp_file, dataset_path, time_series_list, label_list, ids):
    try:
        file_path = os.path.join(dataset_path, slp_file)
        player_1_character = []
        player_2_character = []
        player_1_game_data = []
        player_2_game_data = []
        
        #Finish handle_start
        def handle_start():
        
        def handle_frame():
           

        def parse_game(file_path, port):
            Game(file_path, handlers={Frame: lambda event: handle_frame(event, port)})

        # Parse the game for each player
        game = Game(file_path)
        occupied_ports = [i for i, port in enumerate(game.start.players) if port is not None]
        if len(occupied_ports) != 2:  # Ensure it's a singles game
            return

        for port in occupied_ports:
            parse_game(file_path, port)
            character = game.start.players[port].character.name
            time_series_list.append(np.array(game_data[300: ]))  # Adjust as needed
            label_list.append(1 if character == 'SHEIK' else 0)
            ids.append(slp_file)

    except Exception as e:
        print(f"Error processing {slp_file}: {str(e)}")

In [44]:
def process_slp_file_event_based(slp_file, dataset_path, player_1_game_data, player_2_game_data):
    try:
        file_path = os.path.join(dataset_path, slp_file)
        game_should_be_processed = True  # Flag to determine if the game meets the criteria
        
        player_1_character = []
        player_2_character = []
        player_1_game_data = []
        player_2_game_data = []
        occupied_ports = []
        
        def handle_start(event):
            nonlocal game_should_be_processed
            nonlocal occupied_ports 
            occupied_ports= [i for i, port in enumerate(event.players) if port is not None]
            if len(occupied_ports) != 2:  # Check if it's a singles game
                game_should_be_processed = False
                return  # Skip processing this game
            # Store the characters for player 1 and player 2
            player_1_character.append(event.players[occupied_ports[0]].character.name)
            player_2_character.append(event.players[occupied_ports[1]].character.name)

        def handle_frame(event):
            if not game_should_be_processed:  # Skip processing frames if the game doesn't meet the criteria
                return
            # Process frame data for both players
            for port in occupied_ports:
                frame_data = process_frame_data(event, port)
                if port == occupied_ports[0]:
                    player_1_game_data.append(frame_data)
                else:
                    player_2_game_data.append(frame_data)

        # Parse the game
        handlers = {
            Start: handle_start,
            Frame: lambda event: handle_frame(event),
        }
        Game(file_path, handlers=handlers)

    except Exception as e:
        print(f"Error processing {slp_file}: {str(e)}")


In [69]:
import os
import numpy as np
import pandas as pd
from slippi import Game
from slippi.parse import parse
from slippi.parse import ParseEvent
from joblib import Parallel, delayed
from multiprocessing import Manager
import tqdm

# frames_per_input = 60 * 12  # 12 seconds of gameplay

# One-hot encoding function remains the same
def one_hot_encode(bitmask):
    labels = ['DPAD_LEFT', 'DPAD_RIGHT', 'DPAD_DOWN', 'DPAD_UP', 'Z', 'R', 'L', 'A', 'B', 'X', 'Y', 'START']
    encoded_values = [1, 2, 4, 8, 16, 32, 64, 256, 512, 1024, 2048, 4096]

    # Create a dictionary mapping labels to their encoded values
    label_to_value = dict(zip(labels, encoded_values))

    # Initialize a list to store the one-hot encoded values
    one_hot_encoded = [0] * len(labels)

    # Iterate through labels and set the corresponding one-hot encoded value
    for label, value in label_to_value.items():
        if bitmask & value:
            one_hot_encoded[labels.index(label)] = 1

    return one_hot_encoded

# Handler function to process frame data
def process_frame_data(frame, port):
    buttons = one_hot_encode(frame.ports[port].leader.pre.buttons.physical.value)
    j_x = frame.ports[port].leader.pre.joystick.x
    j_y = frame.ports[port].leader.pre.joystick.y
    c_x = frame.ports[port].leader.pre.cstick.x
    c_y = frame.ports[port].leader.pre.cstick.y
    t_l = frame.ports[port].leader.pre.triggers.physical.l
    t_r = frame.ports[port].leader.pre.triggers.physical.r
    return buttons + [j_x, j_y, c_x, c_y, t_l, t_r]

# Function to process a single SLP file using event-based parsing
def process_slp_file_event_based(slp_file, dataset_path,game_data_list):
    try:
        file_path = os.path.join(dataset_path, slp_file)
        game_should_be_processed = True  # Flag to determine if the game meets the criteria
        player_1_character = []
        player_2_character = []
        player_1_game_data = []
        player_2_game_data = []
        occupied_ports = []
        
        def handle_metadata(event):
            nonlocal game_should_be_processed
            nonlocal occupied_ports 
            occupied_ports = [i for i, port in enumerate(event.players) if port is not None]
            if len(occupied_ports) > 2:  # Check if it's a singles game
                game_should_be_processed = False
                return  # Skip processing this game
            # Store the characters for player 1 and player 2
            player_1_character.append(event.players[occupied_ports[0]].character.name)
            player_2_character.append(event.players[occupied_ports[1]].character.name)

        def handle_frame(event):
            nonlocal game_should_be_processed
            nonlocal occupied_ports 
            if not game_should_be_processed:  # Skip processing frames if the game doesn't meet the criteria
                return
            # Process frame data for both players
            for port in occupied_ports:
                frame_data = process_frame_data(event, port)
                if port == occupied_ports[0]:
                    player_1_game_data.append(frame_data)
                else:
                    player_2_game_data.append(frame_data)

        # Parse the game
        handlers = {
            ParseEvent.METADATA: lambda event: handle_metadata(event),
            Frame: lambda event: handle_frame(event),
        }
        parse(file_path, handlers=handlers)
        # if game_should_be_processed:
        game_data_list.append([player_1_character,player_2_character,player_1_game_data,player_2_game_data])
    except Exception as e:
        print(f"Error processing {slp_file}: {str(e)}")
        
# Example usage with parallel processing
dataset_path = './Slippi_Public_Dataset_v3/'
slp_files = [file for file in os.listdir(dataset_path) if file.endswith('.slp') and 'Sheik' in file and 'Fox' in file]
# print(len(slp_files))

# slp_files = [file for file in os.listdir(dataset_path) if file.endswith('.slp')]

# Initialize multiprocessing lists
manager = Manager()
game_data_list = manager.list()
# label_list = manager.list()
# ids = manager.list()

# Process files in parallel
Parallel(n_jobs=-1, verbose=10)(delayed(process_slp_file_event_based)(slp_file, dataset_path, game_data_list) for slp_file in tqdm.tqdm(slp_files))

# # Convert results to DataFrame
# df = pd.DataFrame({"TimeSeries": list(time_series_list), "Label": list(label_list), "FName": list(ids)})
# df.sort_values(by=['FName', 'Label'], inplace=True)
# df.reset_index(drop=True, inplace=True)
# print(df)
print(game_data_list)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    0.8s

[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    0.9s

[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  65 tasks      | elapsed:    1.3s

[Parallel(n_jobs=-1)]: Done  80 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done  97 tasks      | elapsed:    1.7s

[Parallel(n_jobs=-1)]: Done 114 tasks      | elapsed:    1.9s

[Parallel(n_jobs=-1)]: Done 133 tasks      | elapsed:    2.2s

[Parallel(n_jobs=-1)]: Done 152 tasks      | elapsed:    2.4s

[Parallel(n_jobs=-1)]: Done 173 tasks      | elapsed:    2.7s

[Parallel(n_jobs=-1)]: Done 194 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 217 tasks      | elapsed:    3.2s

[Parallel(n_jobs=-1)]: Done 240 tasks      |

[]


[Parallel(n_jobs=-1)]: Done 2362 out of 2362 | elapsed:   31.8s finished


In [53]:
print(game_data_list)

[]


In [83]:
dataset_path = './Slippi_Public_Dataset_v3/'
slp_files = [file for file in os.listdir(dataset_path) if file.endswith('.slp') and 'Sheik' in file and 'Fox' in file]
# print(len(slp_files))

num_files = 10

occupied_ports = []

def handle_metadata(event):
    # nonlocal occupied_ports
    occupied_ports = [i for i, port in enumerate(event.players) if port is not None]
    print(occupied_ports)
    # occupied_ports = 2
    return occupied_ports

handler = {
    ParseEvent.METADATA: handle_metadata
}
t_1 = time.time()
for slp_file in slp_files[:num_files]:
    file_path = os.path.join(dataset_path, slp_file)
    print(parse(file_path, handler))       
t_2 = time.time()

print(t_2-t_1)

[0, 1]
None
[0, 1]
None
[0, 3]
None
[0, 3]
None
[1, 3]
None
[1, 3]
None
[1, 3]
None
[]
None
[]
None
[]
None
1.2921674251556396


In [87]:
dataset_path = './Slippi_Public_Dataset_v3/'
slp_files = [file for file in os.listdir(dataset_path) if file.endswith('.slp') and 'Sheik' in file and 'Fox' in file]
# print(len(slp_files))

num_files = 10

results = []  # Global list to store results

def handle_metadata(event):
    # Modify the global results list instead of using a local variable
    global results
    occupied_ports = [i for i, port in enumerate(event.players) if port is not None]
    results.append(occupied_ports)  # Append the result for each file

handler = {
    ParseEvent.METADATA: handle_metadata
}

t_1 = time.time()
for slp_file in slp_files[:num_files]:
    file_path = os.path.join(dataset_path, slp_file)
    parse(file_path, handler)
t_2 = time.time()

print(t_2-t_1)
print(results)

1.2872936725616455
[[0, 1], [0, 1], [0, 3], [0, 3], [1, 3], [1, 3], [1, 3], [], [], []]


2.103511333465576
[[0, 1], [0, 1], [0, 3], [0, 3], [1, 3], [1, 3], [1, 3], [], [], []]


In [109]:
dataset_path = './Slippi_Public_Dataset_v3/'
slp_files = [file for file in os.listdir(dataset_path) if file.endswith('.slp') and 'Sheik' in file and 'Fox' in file]
# print(len(slp_files))

num_files = 10


def handle_metadata(event):
    print('e')

handler = {
    ParseEvent.METADATA: handle_metadata
}
t_1 = time.time()
for slp_file in slp_files[:num_files]:
    file_path = os.path.join(dataset_path, slp_file)
    game = Game(file_path)
    ports = [i for i, port in enumerate(game.start.players) if port is not None]
    port_1 = ports[0]
    port_2 = ports[1]
    frames = game.frames
    for i, frame in enumerate(frames):   # Takes all the frames. skips first 5 seconds.
        # frame..ports[0].leader.pre.joystick.x
        frame.ports[port_1].leader.pre.joystick.x
        frame.ports[port_2].leader.pre.joystick.x
t_2 = time.time()

print(t_2-t_1)

3.292348623275757
